In [1]:
# %%html  #to hide the input sections for better view
# <style>
# div.input {
#     display:none;
# }
# </style>

# Dependencies and Setup
import pandas as pd
import numpy as np
import sys
from functools import reduce
from collections import OrderedDict, Counter


In [2]:
# Raw data file
file_to_load = "Resources/purchase_data.csv"

In [3]:
# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data=pd.DataFrame(purchase_data)
purchase_data.head();

In [4]:
### Total Number of Players:
print("### Total Number of Players:")
total_players=pd.DataFrame({"Total Players":purchase_data['SN'].unique().shape[0]},index=["T"])
total_players.head()

### Total Number of Players:


,Total Players
T,576


In [5]:
### Purchasing Analysis (Total):
print("### Purchasing Analysis (Total):")
column_names=["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"]
values=[purchase_data["Item ID"].unique().shape[0],\
        purchase_data["Price"].mean(),\
                purchase_data.shape[0],\
                   purchase_data["Price"].sum()   \
                      ]
aa=[{column_names[i]:values[i] for i in range(4)} ]
#print(aa)
Purchasing_Analysis_Total=pd.DataFrame(aa)
Purchasing_Analysis_Total.rename(index={0:"T"},inplace=True)
pd.set_option('precision',2)
Purchasing_Analysis_Total.head()
#len(purchase_data["Item ID"].value_counts())

### Purchasing Analysis (Total):


,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
T,3.05,780,183,2379.77


In [6]:
###Gender Demographics:
print("###Gender Demographics:")
Gender_data=purchase_data[['SN','Age','Gender']].drop_duplicates() # value_counts()
tot=Gender_data.shape[0]
Gender_data1=Gender_data.groupby("Gender").count()
Gender_data1.rename(columns={"SN":"count","Age":"percentage"},inplace=True)
Gender_data1["percentage"]=Gender_data1["count"]/tot
Gender_data1["percentage"] = pd.Series(["{0:.2f}%".format(val * 100) for val in Gender_data1["percentage"]], index = Gender_data1["percentage"].index)
Gender_data1

###Gender Demographics:


,count,percentage
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [7]:
###Purchasing Analysis (Gender):
print("###Purchasing Analysis (Gender): ('Average Purchase Total per Person by Gender' is abreviated as 'APTpP_Gender')")
Purchase_Gender=purchase_data.groupby(['Gender'])
Gender_groups_count=purchase_data.groupby(['Gender','SN']).size().groupby('Gender').count()
column_names=["PurchaseCount","AveragePrice","TotalValue","APTpP_Gender"]
values=[Purchase_Gender["Purchase ID"].count(),\
         Purchase_Gender["Price"].mean(),\
                Purchase_Gender["Price"].sum(),\
                   Purchase_Gender["Price"].sum()/Gender_groups_count ]
#values=Series.to_frame
values=[value.to_frame() for value in values]
#pd.concat combine list of data_frames to a bigger DataFrame: 
# https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
df_merged_gender = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True), values)
df_merged_gender.columns=column_names
df_merged_gender 

###Purchasing Analysis (Gender): ('Average Purchase Total per Person by Gender' is abreviated as 'APTpP_Gender')


,PurchaseCount,AveragePrice,TotalValue,APTpP_Gender
Gender,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


In [8]:
###Age Demographics
print("###Age Demographics:('Average Purchase Total per Person by Age Group' is abbreviated as 'APTpP_Age')")
#age_bins=[0,10.1,15.1,20,100]
#group_names=["<10","10-14","15-19",">19(adults)"]
age_bins = [0, 9.98, 14.95, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
purchase_data["Age group"]=pd.cut(purchase_data["Age"],age_bins,labels=group_names)
purchase_age=purchase_data.groupby(["Age group"])
age_groups_count=purchase_data.groupby(["Age group","SN"]).size().groupby("Age group").count()

del values
values=[purchase_age["Purchase ID"].count(),\
         purchase_age["Price"].mean(),\
                purchase_age["Price"].sum(),\
                   purchase_age["Price"].sum()/age_groups_count ]
values=[value.to_frame() for value in values]
df_merged_age = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True), values)
df_merged_age.columns=column_names
df_merged_age.rename(columns={"APTpP_Gender":"APTpP_Age"},inplace=True)
#df_merged["APTpP_Age"]=purchase_age["Price"].sum()/age_groups_count
df_merged_age

###Age Demographics:('Average Purchase Total per Person by Age Group' is abbreviated as 'APTpP_Age')


,PurchaseCount,AveragePrice,TotalValue,APTpP_Age
Age group,,,,
<10,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
40+,13,2.94,38.24,3.19


In [9]:
###Top Spenders
print("###Top Spenders:")
Purchase_spender=purchase_data.groupby("SN")
Top_spenders=Purchase_spender["Price"].sum().sort_values(ascending=False)[0:5]

del values
values=[]
for i in range(5):
    #https://stackoverflow.com/questions/22691010/how-to-print-a-groupby-object
   ss=Purchase_spender.get_group(Top_spenders.index[i])["Price"]
   values.append({"Purchase Count":ss.count(),"Average Purchase Price":ss.mean(),"Total Purchase Value":ss.sum()})

df_merged_Top_Spenders=pd.DataFrame(values,index=Top_spenders.index)

df_merged_Top_Spenders


###Top Spenders:


,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Lisosia93,3.79,5,18.96
Idastidru52,3.86,4,15.45
Chamjask73,4.61,3,13.83
Iral74,3.40,4,13.62
Iskadarya95,4.37,3,13.10


In [10]:
###Most Popular Items
print("###Most Popular Items:")
Purchase_item=purchase_data.groupby("Item ID")
Top_items=Purchase_item["Purchase ID"].count().sort_values(ascending=False)[0:5]
del values
values=[]
for i in range(5):
    #https://stackoverflow.com/questions/22691010/how-to-print-a-groupby-object
   ss=Purchase_item.get_group(Top_items.index[i])["Price"]
   ss0=Purchase_item.get_group(Top_items.index[i])["Item Name"]
   ##align the dictionary and produece ordered DataFrame
   #https://stackoverflow.com/questions/44365209/generate-a-pandas-dataframe-from-ordereddict?rq=1
   values.append(OrderedDict({"Item Name":ss0.values[0],"Purchase Count":ss.count(),"item Price":ss.values[0],"TotalPurchaseValue":ss.sum()}))   

col = Counter()
for k in values:
    col.update(k)
df_merged_Top_Items=pd.DataFrame(values,index=Top_items.index,columns=col.keys())
df_merged_Top_Items.sort_values(["Purchase Count","TotalPurchaseValue"],ascending=[False,False])

###Most Popular Items:


,Item Name,Purchase Count,item Price,TotalPurchaseValue
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
92,Final Critic,8,4.88,39.04


In [11]:
###Most Profitable Items
print("###Most Profitable Items")
Purchase_item=purchase_data.groupby("Item ID")
Top_items=Purchase_item["Price"].sum().sort_values(ascending=False)[0:5]
del values
values=[]
for i in range(5):
    #https://stackoverflow.com/questions/22691010/how-to-print-a-groupby-object
   ss=Purchase_item.get_group(Top_items.index[i])["Price"]
   ss0=Purchase_item.get_group(Top_items.index[i])["Item Name"]
   ##align the dictionary and produece ordered DataFrame
   #https://stackoverflow.com/questions/44365209/generate-a-pandas-dataframe-from-ordereddict?rq=1
   values.append(OrderedDict({"Item Name":ss0.values[0],"Purchase Count":ss.count(),"item Price":ss.values[0],"TotalPurchaseValue":ss.sum()}))   
col = Counter()
for k in values:
    col.update(k)
df_merged_Top_Items=pd.DataFrame(values,index=Top_items.index,columns=col.keys())
df_merged_Top_Items

###Most Profitable Items


,Item Name,Purchase Count,item Price,TotalPurchaseValue
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80


Based on the analysis above, we have the following observations:
    1. The number of Male Players are about six times that of female players.
    2. the age group that are mostly involved in this game is 15-30 with a peak around 20-24.
    3. "Oathbreaker, Last Hope of the Breaking Storm" is the most polupar item that the players will buy.